In [1]:
import carla
import cv2
import math
import random
import time
import numpy as np


In [2]:
client = carla.Client('localhost', 2000)
world = client.get_world()

In [3]:
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

In [4]:
vehicle_bp = world.get_blueprint_library().filter('*cybertruck*')
vehicle = world.try_spawn_actor(vehicle_bp[0], spawn_points[0])

In [5]:
CAMERA_POS_Z = 3
CAMERA_POS_X = 0
camera_bp = bp_lib.find('sensor.camera.rgb')
camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x = CAMERA_POS_X))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

In [6]:
camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))
time.sleep(2)
camera.stop()

In [7]:
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [9]:
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()

camera_data = {'image': np.zeros((image_h, image_w, 4))}
camera.listen(lambda image: camera_callback(image, camera_data))

In [ ]:
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])
cv2.waitKey(1)

while True:
    cv2.imshow('RGB Camera', camera_data['image'])

    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()

In [ ]:
vehicle.set_autopilot(True)